In [1]:
import pandas as pd 
import praw
from praw.models import MoreComments

import os

In [5]:
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

In [6]:
# reddit agent to insert credentials
reddit = praw.Reddit(user_agent='Comment Extraction (by /u/Agile_Swordfish_1564)',
                     client_id=client_id,
                     client_secret=client_secret)

In [ ]:
import praw
import pandas as pd


sub = reddit.subreddit("datascience")


posts = []

for submission in sub.top("all"):
    submission.comments.replace_more(limit=None)  # Load all comments
    
    # Get the highest-voted top-level comment
    top_comment = None
    max_upvotes = -1  # Initialize a low vote count

    for comment in submission.comments:
        if hasattr(comment, "body"):  # Ensure it's a valid comment
            if comment.score > max_upvotes:
                max_upvotes = comment.score
                top_comment = comment.body

    # Append to list if a valid answer exists
    if top_comment:
        posts.append({"title": submission.title, "body": top_comment})


df = pd.DataFrame(posts)


df.to_csv("data.csv", index=False)




/tmp/ipykernel_60750/3681945134.py:10: DeprecationWarning: Positional arguments for 'BaseListingMixin.top' will no longer be supported in PRAW 8.
Call this function with 'time_filter' as a keyword argument.
  for submission in sub.top("all"):


                                               title  \
0                                         data siens   
1                            The pain and excitement   
2  Shout Out to All the Mediocre Data Scientists ...   
3                              Let's keep this on...   
4    Guys, we’ve been doing it wrong this whole time   

                                                body  
0  I was once reading this article that went as: ...  
1                     Upper management doesn't care.  
2   Preach. It's okay for your job to just be a job.  
3  Small addendum. Slapping AI / ML on your stati...  
4  Newton & Leibniz would be impressed to see peo...  


In [2]:
df = pd.read_csv("data.csv")
df.head()

,title,body
0,data siens,I was once reading this article that went as: ...
1,The pain and excitement,Upper management doesn't care.
2,Shout Out to All the Mediocre Data Scientists ...,Preach. It's okay for your job to just be a job.
3,Let's keep this on...,Small addendum. Slapping AI / ML on your stati...
4,"Guys, we’ve been doing it wrong this whole time",Newton & Leibniz would be impressed to see peo...


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   100 non-null    object
 1   body    100 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB


In [10]:
qa_pairs = []

for submission in sub.top("all"):
    submission.comments.replace_more(limit=None)  # Load all comments
    
    # Extract highest-voted top-level comment as the best answer
    top_comment = None
    max_upvotes = -1

    for comment in submission.comments:
        if hasattr(comment, "body") and not comment.stickied:  # Ignore stickied comments
            if comment.score > max_upvotes:
                max_upvotes = comment.score
                top_comment = comment.body

    # Store post title and highest-voted answer
    if top_comment:
        qa_pairs.append({"question": submission.title, "answer": top_comment})

    # Now, treat each top-level comment as a potential question
    for comment in submission.comments:
        if hasattr(comment, "body") and not comment.stickied:  # Ignore stickied comments
            max_reply = None
            max_reply_upvotes = -1

            for reply in comment.replies:
                if hasattr(reply, "body") and reply.score > max_reply_upvotes:
                    max_reply_upvotes = reply.score
                    max_reply = reply.body

            if max_reply:  # If a high-voted reply exists, store it as a Q&A pair
                qa_pairs.append({"question": comment.body, "answer": max_reply})

# Convert to DataFrame
df = pd.DataFrame(qa_pairs)

# Save as CSV
df.to_csv("reddit_qa.csv", index=False)

print(df.head())

/tmp/ipykernel_60750/2234435136.py:3: DeprecationWarning: Positional arguments for 'BaseListingMixin.top' will no longer be supported in PRAW 8.
Call this function with 'time_filter' as a keyword argument.
  for submission in sub.top("all"):


                                            question  \
0                                         data siens   
1  I was once reading this article that went as: ...   
2  How do you change your setup on Jupyter to loo...   
3      The reality of 9/10 data scientist job specs!   
4        oh fuck 😂😂😂this had no business to be funny   

                                              answer  
0  I was once reading this article that went as: ...  
1  I’m a ManU fan. I want Cavani to do well. But ...  
2                             IMPORT EXCEL DATAFRAME  
3                                          [deleted]  
4  God the fact that he didn't even refence anoth...  


In [12]:
df = pd.read_csv("reddit_qa.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1517 entries, 0 to 1516
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  1517 non-null   object
 1   answer    1517 non-null   object
dtypes: object(2)
memory usage: 23.8+ KB


In [16]:
df = pd.read_csv("posts.csv")
df.head()

,title,body
0,data siens,NaN
1,I was once reading this article that went as: ...,"['#SportsAnalytics', 'I’m a ManU fan. I want C..."
2,How do you change your setup on Jupyter to loo...,['IMPORT EXCEL DATAFRAME']
3,The reality of 9/10 data scientist job specs!,"['[deleted]', 'It would be...\n\nRequirements:..."
4,oh fuck 😂😂😂this had no business to be funny,"[""God the fact that he didn't even refence ano..."


In [17]:
df.describe()

,title,body
count,12101,12022
unique,12101,4679
top,"Yeah, but it also can reference getting data p...",[]
freq,1,7156


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12101 entries, 0 to 12100
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   12101 non-null  object
 1   body    12022 non-null  object
dtypes: object(2)
memory usage: 189.2+ KB


In [19]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12022 entries, 1 to 12100
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   12022 non-null  object
 1   body    12022 non-null  object
dtypes: object(2)
memory usage: 281.8+ KB
